In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import os
import warnings
warnings.filterwarnings(action='ignore')


import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

In [ ]:
pd.set_option('display.max_columns',None)

In [ ]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

In [ ]:
seed_everything(42)

## 전처리

In [ ]:
submission = pd.read_csv('./다운로드/235801_2021 농산물 가격예측 AI 경진대회/sample_submission.csv')
public_date_list = submission[submission['예측대상일자'].str.contains('2021')]['예측대상일자'].str.split('+').str[0].unique()

In [ ]:
# 데이터 불러오기, 값이 0이면 NAN 변경후 다음날값으로 채우기 없으면 전날값
data = pd.read_csv('real_data.csv')
data = data.replace(0, np.NaN)
data = data.interpolate()
data = data.fillna(method='bfill')
data = data.fillna(method='ffill')

In [ ]:
unique_pum = [
    '배추', '무', '양파', '건고추','마늘',
    '대파', '얼갈이배추', '양배추', '깻잎',
    '시금치', '미나리', '당근',
    '파프리카', '새송이', '팽이버섯', '토마토',
    '청상추', '백다다기', '애호박', '캠벨얼리', '샤인마스캇'
]


In [ ]:
# 필요없는 피쳐 제거
or pum in unique_pum:
    data.drop([f'{pum}_QTYs',f'{pum}_QTYm', f'{pum}_DANQs', f'{pum}_DANQm'], axis=1, inplace=True)
    data.drop(f'{pum}_거래량(kg)', axis=1, inplace=True)

In [ ]:
list_col = list(data.columns[1:169:8])
deal_data = data.iloc[:,1:169:8]

In [ ]:
for unique in unique_pum:
    data.drop([f'{unique}_SAN_NM', f'{unique}_WHSAL_NM', f'{unique}_CMP_NM', f'{unique}_KIND_NM', f'{unique}_LV_NM'], axis=1, inplace = True)

In [ ]:
# 요일, 일, 년도별월, 년도, 주 추출
data['date'] = pd.to_datetime(data['date'])
data['Weekday'] = data['date'].dt.weekday
data['YearMonth'] = data['date'].dt.to_period('M')
data['Year'] = data['date'].dt.to_period('Y')
data['Week'] = data['date'].dt.week

# 단위 별로 데이터 그룹생성
gr_data = data.groupby(data['Week'])
gr_data_day = data.groupby(data['Weekday'])
gr_data_month = data.groupby(data['YearMonth'])
gr_data_year = data.groupby(data['Year'])
gr_data_mon_day = data.groupby([data['YearMonth'], data['Weekday']])

# 단위 별 평균, 합, 표준편차
group_datas = pd.concat([
    gr_data[list_col].mean().rename(columns = lambda x : 'week_means_' + x),
    gr_data[list_col].sum().rename(columns = lambda x : 'week_sum_' + x),
    gr_data[list_col].std().rename(columns = lambda x : 'week_max_' + x)
], axis = 1).reset_index()

group_datas_day = pd.concat([
    gr_data_day[list_col].mean().rename(columns = lambda x : 'day_means_' + x),
    gr_data_day[list_col].sum().rename(columns = lambda x : 'day_sum_' + x),
    gr_data_day[list_col].std().rename(columns = lambda x : 'day_max_' + x)
], axis = 1).reset_index()

group_datas_month = pd.concat([
    gr_data_month[list_col].mean().rename(columns = lambda x : 'month_means_' + x),
    gr_data_month[list_col].sum().rename(columns = lambda x : 'month_sum_' + x),
    gr_data_month[list_col].std().rename(columns = lambda x : 'month_max_' + x)
], axis = 1).reset_index()

group_datas_year = pd.concat([
    gr_data_year[list_col].mean().rename(columns = lambda x : 'year_means_' + x),
    gr_data_year[list_col].sum().rename(columns = lambda x : 'year_sum_' + x),
    gr_data_year[list_col].std().rename(columns = lambda x : 'year_max_' + x)
], axis = 1).reset_index()

group_datas_mon_day = pd.concat([
    gr_data_mon_day[list_col].mean().rename(columns = lambda x : 'mon_day_means_' + x),
    gr_data_mon_day[list_col].sum().rename(columns = lambda x : 'mon_day_sum_' + x),
    gr_data_mon_day[list_col].std().rename(columns = lambda x : 'mon_day_max_' + x)
], axis = 1).reset_index()

# 데이터 모두 합치기
data = pd.merge(data, group_datas, on = 'Week', how = 'left')
data = pd.merge(data, group_datas_day, on = 'Weekday', how = 'left')
data = pd.merge(data, group_datas_month, on = 'YearMonth', how = 'left')
data = pd.merge(data, group_datas_year, on = 'Year', how = 'left')
data = pd.merge(data, group_datas_mon_day, on = ['YearMonth','Weekday'], how = 'left')
data.drop(['Week', 'YearMonth', 'Weekday','Year'], axis=1, inplace=True)

In [ ]:
drop_data = []
# 평균,합,표준편차 값과 농산물가격과 나눠 해당 값들과 영향력있는 퍼센트 피쳐 생성
for col in list_col:
    data[col + 'week_mean'] = data[col] / data['week_means_' + col]
    data[col + 'week_sum'] = data[col] / data['week_sum_' + col]
    data[col + 'week_max'] = data[col] / data['week_max_' + col]
    
    data[col + 'day_mean'] = data[col] / data['day_means_' + col]
    data[col + 'day_sum'] = data[col] / data['day_sum_' + col]
    data[col + 'day_max'] = data[col] / data['day_max_' + col]
    
    data[col + 'month_mean'] = data[col] / data['month_means_' + col]
    data[col + 'month_sum'] = data[col] / data['month_sum_' + col]
    data[col + 'month_max'] = data[col] / data['month_max_' + col]
    
    data[col + 'year_mean'] = data[col] / data['year_means_' + col]
    data[col + 'year_sum'] = data[col] / data['year_sum_' + col]
    data[col + 'year_max'] = data[col] / data['year_max_' + col]
    
    data[col + 'mon_day_mean'] = data[col] / data['mon_day_means_' + col]
    data[col + 'mon_day_sum'] = data[col] / data['mon_day_sum_' + col]
    data[col + 'mon_day_max'] = data[col] / data['mon_day_max_' + col]
    
    # 평균,합,표준편차 값 제거
    drop_data = ['week_means_' + col, 'week_sum_' + col, 'week_max_' + col, 'day_means_' + col, 'day_sum_' + col, 'day_max_' + col, 'month_means_'+col, 'month_sum_'+col,
              'month_max_' + col, 'year_means_' + col, 'year_sum_' + col, 'year_max_' + col, 'mon_day_means_' + col, 'mon_day_sum_' + col, 'mon_day_max_' + col]
    data.drop(drop_data,
              axis=1, inplace=True)

    # 음양의 무한대값이 나왔을경우 다으날 또는 전날값으로 대체
data = data.replace([np.inf, -np.inf], np.NaN)
data = data.fillna(method='bfill')
data = data.fillna(method='ffill')

In [ ]:
col_result = []
col1_count_first = 1
col1_count_two = 4

col2_count_first = 64
col2_count_two = 79

for i in range(len(unique_pum)):
    col1 = data.columns[col1_count_first:col1_count_two].to_list()
    col2 = data.columns[col2_count_first:col2_count_two].to_list()

    col = col1 + col2
    col_result.append(col)
    
    col1_count_first = col1_count_two
    col1_count_two += 3
    
    col2_count_first = col2_count_two
    col2_count_two += 15
    
answer = sum(col_result, [])
answer.insert(0, 'date')
data = data[answer]

## 데이터 정규화

In [ ]:
# minmax
norm = data.iloc[:, 1:].max(0)
data.iloc[:, 1:] = data.iloc[:, 1:] / norm

In [ ]:
# 테스트 데이터
df_test = data.iloc[1781:1811].reset_index(drop=True)
test_df = data.iloc[1811:].reset_index(drop=True)

# 훈련 데이터
data = data.iloc[:1781]
deal_data = data.iloc[:,1:694:18]

# 검증 데이터
df_test_deal_data = df_test.iloc[:,1:694:18]
test_deal_data = test_df.iloc[:,1:694:18]

## 하이퍼파라미터

In [ ]:
device = torch.device("cuda:0")
target_n = 1
learning_rate = 5e-3
BATCH_SIZE = 128
EPOCHS = 15
teacher_forcing = False
n_layers = 2
dropout = 0.2 
window_size = 28
future_size = 28
hidden_dim = 128

In [ ]:
public_date_list

## 추론

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, encoder_input, decoder_input):
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        
    def __len__(self):
        return len(self.encoder_input)
    
    def __getitem__(self, i):
        return {
            'encoder_input' : torch.tensor(self.encoder_input[i], dtype=torch.float32),
            'decoder_input' : torch.tensor(self.decoder_input[i], dtype=torch.float32)
        }
    
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.n_layers = n_layers
        
        self.rnn = nn.GRU(input_dim, hidden_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, inp_seq):
        inp_seq = inp_seq.permute(1,0,2)
        outputs, hidden = self.rnn(inp_seq)
        
        return outputs, hidden

class BahdanauAttention(nn.Module):
    def __init__(self, dec_output_dim, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = nn.Linear(dec_output_dim, units)
        self.W2 = nn.Linear(dec_output_dim, units)
        self.V = nn.Linear(dec_output_dim, 1)

    def forward(self, hidden, enc_output):
        query_with_time_axis = hidden.unsqueeze(1)
        
        score = self.V(torch.tanh(self.W1(query_with_time_axis) + self.W2(enc_output)))
        
        attention_weights = torch.softmax(score, axis=1)
        
        context_vector = attention_weights * enc_output
        context_vector = torch.sum(context_vector, dim=1)

        return context_vector, attention_weights

class Decoder(nn.Module):
    def __init__(self, dec_feature_size, encoder_hidden_dim, output_dim, decoder_hidden_dim, n_layers, dropout, attention):
        super().__init__()
        self.output_dim = output_dim
        self.decoder_hidden_dim = decoder_hidden_dim
        self.n_layers = n_layers
        self.attention = attention
        
        self.layer = nn.Linear(dec_feature_size, encoder_hidden_dim)
        self.rnn = nn.GRU(encoder_hidden_dim*2, decoder_hidden_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, enc_output, dec_input, hidden):
        dec_input = self.layer(dec_input)
        context_vector, attention_weights = self.attention(hidden, enc_output)
        dec_input = torch.cat([torch.sum(context_vector, dim=0), dec_input], dim=1)
        dec_input = dec_input.unsqueeze(0)
        
        output, hidden = self.rnn(dec_input, hidden)
        prediction = self.fc_out(output.sum(0))

        
        return prediction, hidden

    
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, attention):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self, encoder_input, decoder_input, teacher_forcing=False):
        batch_size = decoder_input.size(0)
        trg_len = decoder_input.size(1)
        
        outputs = torch.zeros(batch_size, trg_len-1, self.decoder.output_dim).to(device)
        enc_output, hidden = self.encoder(encoder_input)
        
        dec_input = decoder_input[:, 0] # [:, 0]
        for t in range(1, trg_len):
            output, hidden = self.decoder(enc_output, dec_input, hidden)
            outputs[:, t-1] = output
            if teacher_forcing == True:
                dec_input = decoder_input[:, t]
            else:
                dec_input = output
        
        return outputs
    

def my_custom_metric(pred, true):
    pred = pred[:, [6, 13, 27]]
    true = true[:, [6, 13, 27]]
    target = torch.where(true!=0)
    true = true[target]
    pred = pred[target]
    score = torch.mean(torch.abs((true-pred))/(true))
    
    return score    

def train_step(batch_item, epoch, batch, training, teacher_forcing):
    encoder_input = batch_item['encoder_input'].to(device)
    decoder_input = batch_item['decoder_input'].to(device)
    if training is True:
        model.train()
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            output = model(encoder_input, decoder_input, teacher_forcing)
            loss = criterion(output, decoder_input[:, 1:])
            score = custom_metric(output, decoder_input[:, 1:])
        loss.backward()
        optimizer.step()
        
        return loss, score
    else:
        model.eval()
        with torch.no_grad():
            output = model(encoder_input, decoder_input, False)
            loss = criterion(output, decoder_input[:, 1:])
            score = custom_metric(output, decoder_input[:, 1:])
        return loss, score

def predict(encoder_input):
    model.train()
    encoder_input = encoder_input.to(device)
    decoder_input = torch.zeros([1, future_size+2, target_n], dtype=torch.float32).to(device)
    with torch.no_grad():
        output = model(encoder_input, decoder_input, False)
    return output.cpu()

# 농산물 별로 각각의 모델 추론
pum_len = len(unique_pum)
count = 1
count_f = 19 # 29
for pum in range(pum_len):
    # 모델 불러오기
    save_path = f'./models/{unique_pum[pum]}best_model.pt'
    model = torch.load(save_path)
    model = model.to(device)
    public_date_list = submission[submission['예측대상일자'].str.contains('2021')]['예측대상일자'].str.split('+').str[0].unique()
    outputs = []
    troch_norm = torch.tensor(norm.to_numpy()[count-1])
    for index in range(len(test_df)-1):
        data_df = pd.concat([df_test, test_df.iloc[:index+1]]).iloc[-window_size:].reset_index(drop=True)
        deal_data_df = pd.concat([df_test_deal_data, test_deal_data.iloc[:index+1]]).iloc[-window_size:].reset_index(drop=True)

        data_df = data_df.iloc[:,count:count_f]
        data_df = pd.merge(data_df, deal_data_df, left_index=True, right_index=True, how='left')
        data_df.drop(f'{unique_pum[pum]}_가격(원/kg)_x', axis=1, inplace=True)
        
        encoder_input = torch.tensor(data_df.to_numpy(), dtype=torch.float32)
        encoder_input = encoder_input.unsqueeze(0)
        
        output = predict(encoder_input) * troch_norm
        idx = submission[submission['예측대상일자'].str.contains(public_date_list[index])].index
        # 당일데이터를 사용할수 없기 때문에 전날 데이터로 8, 15, 29일 후 추론값 저장
        submission.loc[idx, f'{unique_pum[pum]}_가격(원/kg)'] = output[0, [7,14,28]].numpy()
    count = count_f
    count_f += 18
submission.to_csv('dacon_inference/dacons.csv', index=False)

In [ ]:
submission.tail(50)

## 학습

In [ ]:
submission_result = pd.read_csv('./다운로드/235801_2021 농산물 가격예측 AI 경진대회/sample_submission.csv')

count_save = 0
count = 1
count_f = 19 # 29

pum_len = len(unique_pum)

for pum in range(pum_len):
    save_path = f'./models/{unique_pum[pum]}best_model.pt'
    x_data = []
    y_data = []
    for i in range(data.shape[0]-window_size-future_size):
        # 원래 피쳐와 기존 농산물가격 결합
        copy_data = pd.merge(data.iloc[:, count:count_f], deal_data, left_index=True, right_index=True, how='left')
        copy_data.drop(f'{unique_pum[pum]}_가격(원/kg)_x', axis=1, inplace=True)
#         x = data.iloc[i:i+window_size, count:count_f].to_numpy() # count:count_f
        x = copy_data.iloc[i:i+window_size, :].to_numpy()
        y = data.iloc[i+window_size:i+window_size+future_size, count: count+1].to_numpy() # count+1 , count +2
        y_0 = np.zeros([1,1])
        x_data.append(x)
        y_data.append(np.concatenate([y_0, y], axis=0))
    x_data = np.array(x_data)
    y_data = np.array(y_data)

    train_test_split = 1
    x_train = x_data[:-train_test_split-future_size]
    y_train = y_data[:-train_test_split-future_size]
    x_val = x_data[-train_test_split:]
    y_val = y_data[-train_test_split:]

    train_dataset = CustomDataset(x_train, y_train)
    val_dataset = CustomDataset(x_val, y_val)

    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=16, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, num_workers=16, shuffle=False)
    sample_batch = next(iter(train_dataloader))

    encoder = Encoder(input_dim=x_data.shape[-1], hidden_dim=hidden_dim, n_layers=n_layers, dropout=dropout)
    attention = BahdanauAttention(dec_output_dim=hidden_dim, units=hidden_dim)
    decoder = Decoder(
        dec_feature_size=target_n, encoder_hidden_dim=hidden_dim, output_dim=target_n,
        decoder_hidden_dim=hidden_dim, n_layers=n_layers, dropout=dropout,
        attention = attention
    )

    model = Seq2Seq(encoder, decoder, attention)
    model = model.to(device)


    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.L1Loss() # mae
    custom_metric = my_custom_metric


    loss_plot, val_loss_plot = [], []
    score_plot, val_score_plot = [], []

    for epoch in range(EPOCHS):
        total_loss, total_val_loss = 0, 0
        total_score, total_val_score = 0, 0

        tqdm_dataset = tqdm(enumerate(train_dataloader))
        training = True
        for batch, batch_item in tqdm_dataset:
            batch_loss, batch_score = train_step(batch_item, epoch, batch, training, teacher_forcing)
            total_loss += batch_loss
            total_score += batch_score

            tqdm_dataset.set_postfix({
                'Epoch': epoch + 1,
                'Loss': '{:06f}'.format(batch_loss.item()),
                'Total Loss' : '{:06f}'.format(total_loss/(batch+1)),
                'Score': '{:06f}'.format(batch_score.item()),
                'Total Score' : '{:06f}'.format(total_score/(batch+1)),
            })
    #     total_loss = total_loss.cpu().numpy()
    #     total_score = total_score.cpu().numpy()
        loss_plot.append(total_loss/(batch+1))
        score_plot.append(total_score/(batch+1))

        tqdm_dataset = tqdm(enumerate(val_dataloader))
        training = False
        for batch, batch_item in tqdm_dataset:
            batch_loss, batch_val_score = train_step(batch_item, epoch, batch, training, teacher_forcing)
            total_val_loss += batch_loss
            total_val_score += batch_val_score

            tqdm_dataset.set_postfix({
                'Epoch': epoch + 1,
                'Val Loss': '{:06f}'.format(batch_loss.item()),
                'Total Val Loss' : '{:06f}'.format(total_val_loss/(batch+1)),
                'Val Score': '{:06f}'.format(batch_val_score.item()),
                'Total Val Score' : '{:06f}'.format(total_val_score/(batch+1)),
            })
        
   
        total_val_loss = total_val_loss.cpu().numpy()
        total_val_score = total_val_score.cpu().numpy()
        val_loss_plot.append(total_val_loss/(batch+1))
        val_score_plot.append(total_val_score/(batch+1))
        
        if np.min(val_loss_plot) == val_loss_plot[-1]:
            torch.save(model, save_path)

    print(f'{unique_pum[pum]}')
    count = count_f
    count_f += 18